In [204]:
from google import genai
from google.genai import types
import base64
import json
import os
from dotenv import load_dotenv

import unicodedata
import re

In [205]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

In [206]:
# the story id, from 1 to 13 for 13 Ainu Kamuy Yukar
SONG_NO = 12

In [207]:
system_instruction_prompt = """You are a professional translator. You know Japanese, English and Chinese. You can translate Japanese into either Chinese or English."""

client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=system_instruction_prompt)],
  )

In [208]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output



In [209]:
poetic_translation_prompt = "Translate the following text from Japanese to Chinese. The original text is a poem. Try to make the translation poetic but keep the original meanings. Display in Traditional Chinese."

descriptive_translation_prompt = "Translate the following text from Japanese to Chinese. Keep the original meanings. Display in Traditional Chinese."

In [210]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [211]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [212]:
# read the Japanese translation of the Ainu original text
with open(f"Chiri_Japanese_Translation/story_translation_{SONG_NO}.txt", "r", encoding="utf8") as f:
    japanese_story = f.read()
    japanese_story = unicodedata.normalize('NFKC', japanese_story)

In [213]:
japanese_story

'獺かわうそが自ら歌った謡\n「カッパ レウレウ カッパ」\n\nカッパ レウレウ カッパ\nある日に,流れに沿うて遊びながら\n泳いで下りサマユンクルの\n水汲路のところに来ると,\nサマユンクルの妹が神の様な美しい容子で\n片手に手桶を持ち片手に\n蒲の束を持って来ているので\n川の縁に私は頭だけ出し,\n「お父様をお持ちですか?\nお母様をお持ちですか?」と云うと,\n娘さんは驚いて眼をきょろきょろさせ\n私を見つけると,怒の色を顔に\n現して,\n「まあ,にくらしい扁平頭,悪い扁平頭が\n人をばかにして.犬たちよ,ココ......」\nと言うと,大きな犬どもが\n駈け出して来て,私を見ると牙を鳴ら\nしている.私はビックリして川の底へ\n潜り込んで直ぐそのまま川底を通って\n逃げ下った.\nそうして,オキキリムイの水汲路の\n川口へ頭だけだして\n見ると,オキキリムイの妹が\n神の様に美しい様子で片手に手桶を持ち\n片手に蒲の束を持って\n来たので私のいうことには,\n「御父様をお持ちですか?\n御母様をお持ちですか?」というと,\n娘さんは驚いて眼をきょろきょろさせ\n私を見ると,怒りの色を顔に\n表して,\n「まあ,にくらしい扁平頭,悪い扁平頭が\n人をばかにして.犬たちよ,ココ......」\nと言うと大きな犬どもが駈け出して来た.\nそれを見て私は先刻の事を思い出し\n可笑しく思いながら川の底へ\n潜りこんで逃げようとしたら,\nまさか犬たちがそんな事をしようとは\n思わなかったのに,牙を鳴らしながら\n川の底まで私に飛び付き\n陸へ私を引き摺り上げ,私の頭も私の体も\n噛みつかれ噛みむしられて,しまいに\nどうなったかわからなくなってしまった.\nふと気が付いて見ると,\n大きな獺の耳と耳の間に私はすわって\nいた.\nサマユンクルもオキキリムイも\n父もなく母もないのを私は知って\nあんな悪戯をしたので罰を当てられ\nオキキリムイの犬どもに殺され\nつまらない死方,悪い死方をするのです.\nこれからの獺たちよ,決して悪戯をしなさるな.\nと,獺が物語った.'

In [214]:
chinese_poetic_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=poetic_translation_prompt)

好的，這是一份將日文詩歌翻譯成繁體中文的譯文，我會盡量保持詩意和原文的含義：

**水獺自歌謠**

「河童 咧嗚咧嗚 河童」

河童 咧嗚咧嗚 河童
某日，順流嬉戲而下，
來到薩瑪雲庫魯汲水之處，
見薩瑪雲庫魯之妹，
容貌若神，美不勝收，
一手提桶，一手持蒲，
我遂於河岸探首，問曰：
「汝有父耶？
汝有母耶？」
少女驚愕，雙目四顧，
見我，怒色浮於面，
「可憎扁頭，惡劣扁頭，
竟敢戲弄於人！犬輩，上啊……」
言罷，巨犬奔騰而至，
呲牙咧嘴，欲噬我身。
我大驚失色，潛入河底，
沿河床倉皇逃遁。

復至奧奇奇里姆伊汲水之口，
探首望之，
見奧奇奇里姆伊之妹，
容貌若神，美不勝收，
一手提桶，一手持蒲，
我遂出言，問曰：
「汝有父耶？
汝有母耶？」
少女驚愕，雙目四顧，
見我，怒色浮於面，
「可憎扁頭，惡劣扁頭，
竟敢戲弄於人！犬輩，上啊……」
言罷，巨犬奔騰而至。
我憶及前事，心生可笑，
欲潛入河底逃遁，
未料犬輩竟有此舉，
呲牙裂嘴，直撲河底，
將我拖拽上岸，
撕咬我頭，抓撓我身，
終至神志不清，不知所終。

恍惚醒來，
竟坐於巨獺雙耳之間。
我明知薩瑪雲庫魯與奧奇奇里姆伊，
皆為無父無母之人，
仍行此惡作劇，
故遭天譴，
為奧奇奇里姆伊之犬所殺，
落得無謂之死，惡劣之死。
告誡後世水獺，切莫惡作劇！
水獺如是說。


In [215]:
chinese_poetic_translation

'好的，這是一份將日文詩歌翻譯成繁體中文的譯文，我會盡量保持詩意和原文的含義：\n\n**水獺自歌謠**\n\n「河童 咧嗚咧嗚 河童」\n\n河童 咧嗚咧嗚 河童\n某日，順流嬉戲而下，\n來到薩瑪雲庫魯汲水之處，\n見薩瑪雲庫魯之妹，\n容貌若神，美不勝收，\n一手提桶，一手持蒲，\n我遂於河岸探首，問曰：\n「汝有父耶？\n汝有母耶？」\n少女驚愕，雙目四顧，\n見我，怒色浮於面，\n「可憎扁頭，惡劣扁頭，\n竟敢戲弄於人！犬輩，上啊……」\n言罷，巨犬奔騰而至，\n呲牙咧嘴，欲噬我身。\n我大驚失色，潛入河底，\n沿河床倉皇逃遁。\n\n復至奧奇奇里姆伊汲水之口，\n探首望之，\n見奧奇奇里姆伊之妹，\n容貌若神，美不勝收，\n一手提桶，一手持蒲，\n我遂出言，問曰：\n「汝有父耶？\n汝有母耶？」\n少女驚愕，雙目四顧，\n見我，怒色浮於面，\n「可憎扁頭，惡劣扁頭，\n竟敢戲弄於人！犬輩，上啊……」\n言罷，巨犬奔騰而至。\n我憶及前事，心生可笑，\n欲潛入河底逃遁，\n未料犬輩竟有此舉，\n呲牙裂嘴，直撲河底，\n將我拖拽上岸，\n撕咬我頭，抓撓我身，\n終至神志不清，不知所終。\n\n恍惚醒來，\n竟坐於巨獺雙耳之間。\n我明知薩瑪雲庫魯與奧奇奇里姆伊，\n皆為無父無母之人，\n仍行此惡作劇，\n故遭天譴，\n為奧奇奇里姆伊之犬所殺，\n落得無謂之死，惡劣之死。\n告誡後世水獺，切莫惡作劇！\n水獺如是說。\n'

In [216]:
chinese_descriptive_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=descriptive_translation_prompt)

以下是將日文文本翻譯成繁體中文的結果：

**水獺自唱的歌謠**
「河童 咧嗚咧嗚 河童」

河童 咧嗚咧嗚 河童
有一天，我沿著河流嬉戲，
游到薩瑪雲庫魯的
汲水路口時，
看見薩瑪雲庫魯的妹妹，像神一般美麗，
一手提著水桶，一手
拿著香蒲束走來，
我便只露出頭在河邊，
問道：「妳有父親嗎？
妳有母親嗎？」
那女孩嚇了一跳，眼睛四處張望，
發現了我，臉上露出
憤怒的表情，
說：「啊，可惡的扁頭，壞扁頭
竟敢愚弄人。狗兒們，過來......」
話音未落，大狗們就
衝了過來，對著我齜牙咧嘴。
我嚇了一跳，趕緊潛入河底，
直接沿著河底逃走了。
然後，我又在奧基基里穆伊的汲水路口
只露出頭來
看見奧基基里穆伊的妹妹，
像神一般美麗，一手提著水桶，
一手拿著香蒲束走來，
我就問她：
「妳有父親嗎？
妳有母親嗎？」
那女孩嚇了一跳，眼睛四處張望，
發現了我，臉上露出
憤怒的表情，
說：「啊，可惡的扁頭，壞扁頭
竟敢愚弄人。狗兒們，過來......」
話音未落，大狗們就衝了過來。
我看到這情景，想起剛才的事，
覺得好笑，正想潛入河底
逃走，
沒想到狗兒們竟然會做出那種事，
牠們齜牙咧嘴地
跳入河底，撲向我，
把我拖到岸上，我的頭和身體
都被牠們咬住、撕扯，最後
發生了什麼事，我就不知道了。
突然回過神來，
發現自己正坐在
一隻大水獺的兩隻耳朵之間。
我知道薩瑪雲庫魯和奧基基里穆伊
都沒有父母，
我還那樣惡作劇，所以受到了懲罰，
被奧基基里穆伊的狗兒們殺死，
落得一個無聊的死法，糟糕的死法。
今後的水獺們啊，千萬不要再惡作劇了。
水獺這樣講述著。


In [217]:
# write the Chinese translations to Markdown file

#read in the template
with open("templates/raw_output_md_template", "r", encoding="utf8") as f:
    md_template = f.read()
    md_template = unicodedata.normalize('NFKC', md_template)

In [218]:
md_output = md_template.format(translated_language="Chinese", ainu_title=ainu_titles[SONG_NO - 1], 
                               japanese_title=japanese_titles[SONG_NO], input_japanese = japanese_story, output_poetic=chinese_poetic_translation, output_descriptive=chinese_descriptive_translation)

In [219]:
md_output

'# LLM-assisted Translation project of Ainu shin\'yoshu (アイヌ神謡集) collected by Chiri Yukie 知里幸惠 (1903 - 1922).\n\n## Chinese Translation of\n\n## Esaman yaieyukar, “Kappa reureu kappa” 獺かわうそが自ら歌った謡「カッパ レウレウ カッパ」\n\n### Model\ngemini-2.0-flash-001\n\n### Setting\n```\ngenerate_content_config = types.GenerateContentConfig(\n    temperature = 0,\n    top_p = 0,\n    max_output_tokens = 8192,\n    response_modalities = ["TEXT"],\n    safety_settings = [types.SafetySetting(\n      category="HARM_CATEGORY_HATE_SPEECH",\n      threshold="OFF"\n    ),types.SafetySetting(\n      category="HARM_CATEGORY_DANGEROUS_CONTENT",\n      threshold="OFF"\n    ),types.SafetySetting(\n      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",\n      threshold="OFF"\n    ),types.SafetySetting(\n      category="HARM_CATEGORY_HARASSMENT",\n      threshold="OFF"\n    )],\n    system_instruction=[types.Part.from_text(text=si_text1)],\n  )\n```\n\n### System prompt\n```\nYou are a professional translator. You know Japanes

In [220]:
# setup the output file name

s = ainu_titles[SONG_NO - 1].split()
md_name_part = s[0]

name_2nd_part = ""

for text in s:
    if text.startswith('“'):
        name_2nd_part = text.replace('“', '').replace('”', '')
        
md_name_part += "_" + name_2nd_part

In [221]:
with open(f"LLM_prompts_and_raw_outputs/{SONG_NO}_{md_name_part}_to_Chinese.md", "w", encoding="utf8") as f:
    f.write(md_output)